# First steps

Expressions will evalaute and print the result

In [1]:
1 + 3

4

```Console.Write``` and ```Console.WriteLine``` will also display output

In [2]:
Console.WriteLine("hello");
Console.WriteLine("world");

hello


world


To display output use the ```display``` api, that uses advanced formatting to print to screen

In [3]:
display("hello world");

hello world

try it with collections 

In [4]:
display(new []{"hello", "world"});

index,value
0,hello
1,world


classes will be printed as table

In [5]:
class Person {
 public string Name {get;set;}
 public string Surname {get;set;}
 public int Age {get;set;}
}

display(new Person{ Name = "Mitch", Surname = "Buchannon", Age = 42});

Name,Surname,Age
Mitch,Buchannon,42


and so will be object collections

In [6]:
var groupOfPeople = new [] {
    new Person{ Name = "Mitch", Surname = "Buchannon", Age = 42},
    new Person{ Name = "Hobie ", Surname = "Buchannon", Age = 23},
    new Person{ Name = "Summer", Surname = "Quinn", Age = 25},
    new Person{ Name = "C.J.", Surname = "Parker", Age = 23},
};

display(groupOfPeople);

index,Name,Surname,Age
0,Mitch,Buchannon,42
1,Hobie,Buchannon,23
2,Summer,Quinn,25
3,C.J.,Parker,23


Note that dictionaries will have the key printed as well

In [7]:
display(groupOfPeople.ToDictionary(p => $"{p.Name}--{p.Surname}"));

key,Name,Surname,Age
Mitch--Buchannon,Mitch,Buchannon,42
Hobie --Buchannon,Hobie,Buchannon,23
Summer--Quinn,Summer,Quinn,25
C.J.--Parker,C.J.,Parker,23


# Advanced display usage

the ```display``` function returns an object that can be used to update the output at a later time

In [8]:
var fruitOutput = display("let's get some fruit!");
var basket = new [] {"apple", "orange", "coconut", "pear", "peach"};
foreach (var fruit in basket){
    System.Threading.Thread.Sleep(1000);
    fruitOutput.Update($"And I have {fruit}");    
}
System.Threading.Thread.Sleep(1000);
fruitOutput.Update($"So i got : {string.Join(", ", basket)}");

So i got : apple, orange, coconut, pear, peach

There is a dom api that can be used to emit html for reach display

In [9]:
var mitch = groupOfPeople.First(p => p.Name == "Mitch");
display(div(span(a[href: @"https://en.wikipedia.org/wiki/Mitch_Buchannon"](b(mitch.Name)),i(mitch.Age))));

Mitch 42

Now we can use the dom api and ```Formatter``` class to define a custom formatter we want to bind to the Person type

In [10]:
Formatter<Person>.Register( (person, writer ) => {
    writer.Write(div(span(b(person.Name),i(person.Age))));
}, "text/html");

display(mitch);

Mitch 42

The ```Register``` method asks for an function that will accept an instance of the type registered and a writer where we will emit the formatted value. Note the second parameter required by the ```Register``` api is a string that idenfities the mimetype of the formatted output. In jupypter client ```"text/plain"``` and ```"text/html"``` are natively supported

# Using external libraries

It is possible to load assemblies using a ```#r "[path to dll]"``` directive with the path to the dll to load, but is also possible to use the directive to import a nuget package.
To load the latest version of a package just use ```#r "nuget:[package name]```, to lock the notebook on a specific version then use ```#r "nuget:[package name], 1.2.3```.

Let's use all we have seen so far then!

Load ```System.Reacive.Linq``` at version 4.1.5 and import namespaces

In [11]:
#r "nuget:System.Reactive.Linq, 4.1.5"

using System.Reactive;
using System.Reactive.Linq;
using System.Reactive.Concurrency;

In [12]:
var peopleDisplay = display(new Person{Name = "N/A", Surname = "N/A", Age = 0});

var sub = Observable
    .Interval(TimeSpan.FromSeconds(3),CurrentThreadScheduler.Instance)    
    .Take(groupOfPeople.Length)
    .Select(i => groupOfPeople[i])    
    .Subscribe(person => peopleDisplay.Update(person));

C.J. 23